# Agents in practice

In this notebook we will first see how the agent works in practice.
LangChain comes with a set of predefined agents that can be used with tools of our choice.

We will start with `AgentType.OPENAI_FUNCTIONS` - it runs as long as model's output `finish_reason = function_call` and includes prevoius function results into the next round prompt. One disadvantage of such approach is that it doesn't plan ahead - it only decides on (calling a function / finishing with final sentence) based on the previous function results. 

Next step will be to overcome those pitfalls and implement it from scratch to see how it works.
It will give you a thorough understanding of current possibilities but also limitations.

### More tools

To make it more interesting, apart from using our simple calculator function to add two numbers, we will provide the model with web search to enable it to fetch missing information.

One caveat is that the web search does not always return results in nice format, usually it is a concatenation of many different information sources. 

We will use `GoogleSerperAPIWrapper` which is a wraper for Google search and gives much better results than free `DuckDuckGoSearchRun`.
One additional setup step here is to create free account with 2500 free requests. In only requires confirming the email.
Go to https://serper.dev , sing up, confirm email and copy the API key.
Place in in .env file with name `SERPER_API_KEY` or enter it directly in the cell belowe.
We will modify it slightly to reduce results length to just include first 250 characters. Longer results will only distract the model.

If you want, you can also register for free tier for Wolfram Alpha at https://developer.wolframalpha.com/ and create Simple App and copy its ID to .env file as `WOLFRAM_ALPHA_APPID`. This one is not necessary for the learning purposes but it will make the results more interesting.
It gives you 2000 free requests per month. 

In [5]:
import os
import dotenv
import getpass

from langchain.utilities import GoogleSerperAPIWrapper, DuckDuckGoSearchAPIWrapper

dotenv.load_dotenv()
if os.getenv("SERPER_API_KEY") is None:
    os.environ["SERPER_API_KEY"] = getpass.getpass("Serper API Key not set! Enter Serper API Key: ")
if os.getenv("WOLFRAM_ALPHA_APPID") is None:
    print("Wolfram Alpha App ID not set!")

# gl = country code
search = GoogleSerperAPIWrapper(gl='lt')
duck_duck_go = DuckDuckGoSearchAPIWrapper(region='lt')

def search_run(input):
    # return duck_duck_go.run(input)[:250]
    return search.run(input)[:250]

print(search_run("What is the current weather in Vilnius?"))

-1°C


The results in this notebook will for sure vary each rerun depending on the search results.

In [28]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool, Tool
from langchain.tools import format_tool_to_openai_function

@tool
def add_two_number(x, y):
    """Function that adds two numbers"""
    return x + y

@tool 
def multiply_two_number(x, y):
    """Function that multiplies two numbers"""
    return x * y

tools = [
    Tool(
        name="web_search", # note that we can't use spaces with OpenAi function names!
        func=search_run,
        description="Usefull if you need to find out some additional information. You should ask targeted questions."
    ),
    add_two_number,
    multiply_two_number
]

agent = initialize_agent(
    tools=tools, # type: ignore
    llm=ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=0.),
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True,
    max_iter=2,
    handle_parsing_errors=True,
)

In [23]:
output = agent.invoke("What is the sum of populations of France and Germany as of 2023?") # type: ignore



> Entering new AgentExecutor chain...

Invoking: `web_search` with `population of France 2023`


The current population of France in 2023 is 64,756,584, a 0.2% increase from 2022. The population of France in 2022 was 64,626,628, a 0.15% increase from 2021. The population of France in 2021 was 64,531,444, a 0.08% increase from 2020.
Invoking: `web_search` with `population of Germany 2023`


Germany's population in 2023 Germany's total population was 83.31 million in January 2023. Data shows that Germany's population decreased by 114 thousand (-0.1 percent) between 2022 and 2023. 50.7 percent of Germany's population is female, while 49.3
Invoking: `add_two_number` with `{'x': 64756584, 'y': 83310000}`


148066584The sum of the populations of France and Germany as of 2023 is 148,066,584.

> Finished chain.


In [24]:
output

{'input': 'What is the sum of populations of France and Germany as of 2023?',
 'output': 'The sum of the populations of France and Germany as of 2023 is 148,066,584.'}

Let's check some other query:

In [25]:
output = agent.invoke("What is the price of 123 Tesla stocks?") # type: ignore



> Entering new AgentExecutor chain...

Invoking: `web_search` with `price of 123 Tesla stocks`


Discover historical prices for TSLA stock on Yahoo Finance. View daily, weekly or monthly format back to when Tesla, Inc. stock was issued. Missing: 123 | Show results with:123. Tesla Inc stocks price quote with latest real-time prices, charts, finanI'm sorry, but I couldn't find the specific price of 123 Tesla stocks. However, you can check the current price of Tesla stocks on financial websites or stock market platforms.

> Finished chain.


In [10]:
output

{'input': 'What is the price of 123 Tesla stocks?',
 'output': "I'm sorry, but I couldn't find the specific price of 123 Tesla stocks. However, you can check the current price of Tesla stocks on financial websites or stock market platforms."}

As you can see, the model didn't came up with an idea to first check the price for one stock and then multiply it appropriately.

In [27]:
output = agent.invoke("What is the price of two Tesla stocks plus the price of three Microsoft stocks?") # type: ignore



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'current price of Tesla stock'}`


242.84 +5.43 (2.29%)
Invoking: `web_search` with `{'query': 'current price of Microsoft stock'}`


369.67 +0.15 (0.04%)Could not parse tool input: {'name': 'add_two_number', 'arguments': '{\n  "x": 242.84 * 2,\n  "y": 369.67 * 3\n}'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse tool input: {'name': 'add_two_number', 'arguments': '{\n  "x": 242.84 * 2,\n  "y": 369.67 * 3\n}'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse tool input: {'name': 'add_two_number', 'arguments': '{\n  "x": 242.84 * 2,\n  "y": 369.67 * 3\n}'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse tool input: {'name': 'add_two_number', 'arguments': '{\n  "x": 242.84 * 2,\n  "y": 369.67 * 3\n}'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse to

KeyboardInterrupt: 

In [38]:
@tool
def add_two_number(x, y):
    """Always use this function to add numbers. Never perform addition manually! Inputs must be numbers not equations!"""
    return x + y

@tool 
def multiply_two_number(x, y):
    """Always use this function to multiply numbers. Never perform multiplication manually! Inputs must be numbers not equations!"""
    return x * y

tools = [
    Tool(
        name="web_search", # note that we can't use spaces with OpenAi function names!
        func=search_run,
        description="Usefull if you need to find out some additional information. You should ask targeted questions."
    ),
    add_two_number,
    multiply_two_number
]

# new model gpt-3.5-turbo-1106
# but there are complaints https://community.openai.com/t/new-gpt-3-5-turbo-1106-api-is-horrific/496732
agent = initialize_agent(
    tools=tools, # type: ignore
    llm=ChatOpenAI(model='gpt-3.5-turbo-1106', temperature=0.),
    # llm=ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=0.),
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True,
    max_iter=2,
    handle_parsing_errors=True,
)

In [41]:
output = agent.invoke("What is the price of two Tesla stocks plus the price of three Microsoft stocks?") # type: ignore



> Entering new AgentExecutor chain...

Invoking: `web_search` with `current price of Tesla stock`


242.84 +5.43 (2.29%)
Invoking: `multiply_two_number` with `{'x': 2, 'y': 242.84}`


485.68
Invoking: `web_search` with `current price of Microsoft stock`


369.67 +0.15 (0.04%)
Invoking: `multiply_two_number` with `{'x': 3, 'y': 369.67}`


1109.01
Invoking: `add_two_number` with `{'x': 485.68, 'y': 1109.01}`


1594.69The price of two Tesla stocks plus the price of three Microsoft stocks is $1594.69.

> Finished chain.


But there were also some problems:
```
Could not parse tool input: {'name': 'add_two_number', 'arguments': '{"x":(242.84*2),"y":(369.67*3)}'} because the `arguments` is not valid JSON
```
or

`Invoking: `add_two_number with `{'x': 2, 'y': 3}`

Solution:

Here we can work around it by using more powerfull calculator: WolframAlpha API.
It can handle arbitrary valid calculations. Alternative solution would be to write cusom function that would parse the input and calculate it.

A very easy but dangerous one would be to provide the model with Python `eval` function. It would be able to calculate any valid Python expression but it would also be able to execute any Python code. It would be a huge security risk.

But in general, remember that `OpenAI` function calling can hallucinate and produce invalid JSONs.

In [49]:
dotenv.load_dotenv()
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper

tools = [
    Tool(
        name="web_search",
        func=search_run,
        description="Usefull if you need to find out some additional information. You should ask targeted questions."
    ),
    Tool(
        name="calculator", 
        func=WolframAlphaAPIWrapper().run, # type: ignore
        description="Calculator. Use it for any calculation. Input should be a string with equation."
    ),
]

agent = initialize_agent(
    tools=tools, # type: ignore
    llm=ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=0.),
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True,
    max_iter=2,
    handle_parsing_errors=True,
)

In [51]:
output = agent.invoke("What is the price of two Tesla stocks plus the price of three Microsoft stocks?") # type: ignore



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'current price of Tesla stock'}`


242.84 +5.43 (2.29%)
Invoking: `web_search` with `{'query': 'current price of Microsoft stock'}`


369.67 +0.15 (0.04%)
Invoking: `calculator` with `{'equation': '2 * 242.84 + 3 * 369.67'}`


Assumption: 2×242.84 + 3×369.67 
Answer: 1594.69The price of two Tesla stocks plus the price of three Microsoft stocks is $1594.69.

> Finished chain.


# Build the agent

Now we will build the agent from scratch. Instead of using function calling directly thourgh `OpenAi API`, we will prompt the model (and hope it comply :) ) to produce valid function arguments as output.

When using function calling, the only output we get when the model decides to call a functions are its arguments. We do not get any explanation why it decided to call it.
To make it better, we can include additional `Think` step before `Act` as suggested in [ReAct paper](https://react-lm.github.io). 
It gives the LLM a chance to explain its decision and as a result leads to better outcomes.

#### Providing actions results to model

The model prompt will follow this pattern:
```
general instructions

QUESTION

steps (possibly many):
Thought: I need to fetch some information about X from the internet
Action: web_search
Action Input: "What is X"
Observation: "X is ..."

and at the end:
Thought: I now know the final answer
Final Answer: the final answer to the original input question
``` 

The `observation` will be the output of the action.
In each step, the model will generate Thought, Action and Action Input (or Final Answer if it is the end) and we will run it localy and provide it with the result.

To do so we can take advantage of optionally stoping the LLM from further generation if it produces a specific string, in this case `Observation: `.
Later we will extend the prompt with action results and run it again.

In [52]:
# Necessary imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage, BaseMessage
import re
from functools import reduce

To make thinks simpler, we won't use JSON for forcing the model to produce structured output.
The `OpenAI` models are powerful enought to spot the pattern in the prompt and follow it, of course to some extent.

And because of that, the action results will be provided as a string, not a JSON object. We need to change the tools to work with that.

In [54]:
@tool
def add_numbers(input: str):
    """Usefull if you need to add numbers e.g 2+3. Input should be in the format `x+y+...`"""
    return sum([float(x) for x in input.split("+")])

@tool 
def multiply_numbers(input: str):
    """Usefull if you need to multiply numbers e.g 2*3. Input should be in the format `x*y*...`"""
    return reduce(lambda x, y: x * y, [float(x) for x in input.split("*")])

tools2 = [
    Tool(
        name="web_search",
        func=search_run,
        description="Usefull if you need to find out some additional information. You should ask targeted questions."
    ),
    add_numbers,
    multiply_numbers
]

#### Agent prompt template

There are many prompt tempales for agent. We will work with the simple short one.
If you want to search for more complex prompts the [LangChain Hub](https://smith.langchain.com/hub?organizationId=8a005f7e-7bfc-5a51-856d-a98ed4e65f71) has many of them, some follow JSON format some don't.

We also need to write custom prompt formatting to meed our defined structure.

In [64]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format. When you know the answer, write it as `Final Answer: `

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> List[BaseMessage]:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log # this contains Thought, Action and Action Input
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools2,
    input_variables=["input", "intermediate_steps"]
)

The `input_variables=["input", "intermediate_steps"]` might look confusing at first, because we only specify part of whats is in the prompt.

`input` and `intermediate_steps` will be passed to `format_messages` as `kwargs` everything else will be set in this function. Later `pop` will remove `intermediate_steps` from kwargs and in the following lines we will add what is missing in the template.

#### Parsing LLM output

If the models follows the patters for thought, action, observation etc. at each step we will either get the output
```
Thought: ...
Action: ...
Action Input: ...
Observation:| <- here ends the model output because we will use stop criteria
```

or

```
Thought: I now know the final answer
Final Answer: ...
```

So we need to write a custom parser to detect those two situations and extract the results.

In [65]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # According to LangChain docs:
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={
                    "output": llm_output.split("Final Answer:")[-1].strip() # <- here we take what is after the final answer marker
                },
                log=llm_output,
            )
        # Parse out the action and action input
        # this regular expression matches Action: and Action Input: lines
        # If you don't fully understand it, don't worry, it's not that important, usually you can google/chat-gpt what you need
        # If you want to get a better feeling about it, you can explore it here: https://regex101.com/r/dTDhMy/1
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

### Set up the agent

Now we have all necessary building blocks to create our agent.
We will first use predefined class `LLMSingleActionAgent`. Single coresponds to fact that it can only produce one action at a time, not a sequence of actions.

We set `stop=["\nObservation:"]` which will be sent with prompt to `OpenAI API` and will stop the generation when this string is produced.

In [66]:
llm = ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=0)

llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools2]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    # allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools2, verbose=True)

In [67]:
agent_executor.run("What is the sum of populations of France and Germany as of 2023?")



> Entering new AgentExecutor chain...
Thought: I need to find the populations of France and Germany in 2023 and then add them together.
Action: web_search
Action Input: "population of France in 2023"

Observation:The current population of France in 2023 is 64,756,584, a 0.2% increase from 2022. The population of France in 2022 was 64,626,628, a 0.15% increase from 2021. The population of France in 2021 was 64,531,444, a 0.08% increase from 2020.
I have found the population of France in 2023. Now I need to find the population of Germany in 2023.
Action: web_search
Action Input: "population of Germany in 2023"

Observation:Germany 2023 population is estimated at 83,294,633 people at mid year. Germany population is equivalent to 1.04% of the total world population. Germany ranks number 19 in the list of countries (and dependencies) by population. The population density 
I have found the population of Germany in 2023. Now I can add the populations of France and Germany together.
Action: 

'The sum of the populations of France and Germany as of 2023 is 148,051,217.'

In [68]:
agent_executor.run("What is the price of 123 Tesla stocks?")



> Entering new AgentExecutor chain...
Thought: I need to find the price of 123 Tesla stocks.
Action: web_search
Action Input: "Tesla stock price"

Observation:242.84 +5.43 (2.29%)
I have found the current price of Tesla stocks.
Action: multiply_numbers
Action Input: "123*242.84"

Observation:29869.32
I now know the final answer.
Final Answer: The price of 123 Tesla stocks is $29,869.32.

> Finished chain.


'The price of 123 Tesla stocks is $29,869.32.'

### Thoughts and observations

As you can see, adding `Thought` step makes model predictions much better.
It now knows that it should check the price of one stock and then multiply it by the number of stocks.

There are many [tools](https://python.langchain.com/docs/integrations/tools) and [toolkits](https://python.langchain.com/docs/integrations/toolkits) implemented in LangChain which you can check out but the idea is the same as whit simple web search or Python function call.
You must provide the model with its names and description of what it does and how to use it.
For example:
- `WikipediaQueryRun` <- gets the top matching wikipedia article, but becareful, it fetches the whole article which can be very long and fill up the prompt quickly.
- [You search](https://you.com) <- responses optimized for LLM
- [Tavily search](https://app.tavily.com/) <- includes sources in response, suggests follow-up queries

Quick tips to improve the agent:
- add current date to the prompt at runtime
- add user location to the prompt at runtime